In [1]:
import numpy as np
import pandas as pd
from pandas import datetime
import math
from tensorflow.contrib.rnn import LSTMCell, DropoutWrapper, MultiRNNCell
from tensorflow.contrib.layers import fully_connected
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import json
%matplotlib inline

In [50]:
class PRSA_lstm:
    
    def __init__(self, model_to_restore="lstm-200"):
    
        self._input_col_cfg = ['pm25_prev', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir', 'NE', 'NW', 'SE', 'cv']
        self._time_steps = 23
        self._X_min, self._X_max = (np.array([  0.00000000e+00,  -4.00000000e+01,  -1.90000000e+01,
                              9.91000000e+02,   4.50000000e-01,   0.00000000e+00,
                              0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                              0.00000000e+00,   0.00000000e+00]),
                             np.array([  9.94000000e+02,   2.80000000e+01,   4.20000000e+01,
                                      1.04600000e+03,   5.65490000e+02,   2.70000000e+01,
                                      3.60000000e+01,   1.00000000e+00,   1.00000000e+00,
                                      1.00000000e+00,   1.00000000e+00]))
        tf.reset_default_graph()
        self._sess = tf.Session()
        new_saver = tf.train.import_meta_graph(meta_graph_or_file="{}.meta".format(model_to_restore))
        new_saver.restore(self._sess, model_to_restore)
        self.graph = self._sess.graph
                
   
    
    def _preprocess(self, data):
        assert(np.all(data.columns.values == np.array(['No', 'year', 'month', 'day', 'hour', 'pm2.5', 'DEWP', 'TEMP',
           'PRES', 'cbwd', 'Iws', 'Is', 'Ir'])))
        
        assert(len(data)==self._time_steps+1)
        assert(np.all(np.isnan(data["pm2.5"].iloc[0:-1])==False))
        assert(np.isnan(data["pm2.5"].iloc[-1]))
        
        data.index = data.apply(lambda x:datetime(x.year, x.month, x.day, x.hour), axis=1)
        data.drop(labels=["year", "month", "day", "hour"], axis=1, inplace=True)
       
        data["pm2.5"] = data["pm2.5"].shift(1)
        data.drop(index=data.index[0], inplace=True)
        assert(np.all(np.isnan(data["pm2.5"])==False))
        data.rename(columns={"pm2.5":"pm25_prev"}, inplace=True)
        data = pd.concat([data, pd.get_dummies(data.cbwd)], axis=1)
        data.drop(labels=["cbwd", "No"], axis=1, inplace=True)
     
        ts = pd.date_range(start=data.index.min(), end=data.index.max(), freq="H")
        data = data.reindex(index=ts, method="pad")
      
        cols_to_add = set(self._input_col_cfg) - set(data.columns)
        for c in cols_to_add:
            data[c] = None
        
        data = data[self._input_col_cfg]
        data.fillna(0, inplace=True)
        assert(data.shape[0]==self._time_steps)
        X_train = (data.values-self._X_min)/(self._X_max-self._X_min)
        
        return(X_train)
    
    def predict(self, data):
        data = self._preprocess(data)
        
     
    def __exit__(self):
        
        self._sess.close()
        tf.reset_default_graph()
        

In [51]:
data = pd.read_csv("PRSA_data_2010.1.1-2014.12.31.csv")
data = data[data["pm2.5"].isna()==False]

m = PRSA_lstm()
data = data.iloc[0:24]
data.at[data.index[-1],"pm2.5"] = None
m._preprocess(data)

INFO:tensorflow:Restoring parameters from lstm-200


array([[0.12977867, 0.36764706, 0.24590164, 0.52727273, 0.00394662,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.14889336, 0.42647059, 0.2295082 , 0.54545455, 0.00552173,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.15995976, 0.48529412, 0.2295082 , 0.56363636, 0.00868965,
        0.03703704, 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.18209256, 0.48529412, 0.2295082 , 0.56363636, 0.01026476,
        0.07407407, 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.138833  , 0.48529412, 0.21311475, 0.56363636, 0.01183987,
        0.11111111, 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.10965795, 0.48529412, 0.21311475, 0.58181818, 0.01500779,
        0.14814815, 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.1056338 , 0.48529412, 0.2295082 , 0.6       , 0.

In [75]:
tf.reset_default_graph()

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph(meta_graph_or_file="lstm-200.meta")
    new_saver.restore(sess, "lstm-200")
    print(sess.graph.get_tensor_by_name("Placeholder:0"))


INFO:tensorflow:Restoring parameters from lstm-200
Tensor("Placeholder:0", shape=(50, 23, 11), dtype=float32)
